In [1]:
import pandas as pd
import numpy as np
import datetime

# this dir should point at where ever your sanofi Mclaren directory is syncing
# dir = "C:/Users/mark_/Sanofi/Sanofi x McLaren sharing - General/Race Dashboard data/"

from sys import platform
# if platform == "linux" or platform == "linux2":
    # linux    
if platform == "darwin":
    # OS X
    dir = "/Users/mark/Documents/GitHub/IP21/"
elif platform == "win32":
    # Windows...
    # dir = "C:/Users/mark_/Documents/McLaren2021/Sanofi/IP21/"
    # trying the microsoft teams sanofi share folder
    dir = "C:/Users/mark_/Sanofi/Sanofi x McLaren sharing - General/Frankfurt sprint/SFD/OEE data/"

    


In [2]:
# read Detailed Events file
file = (dir + 'AL6_Packaging_Detailed_Events.xlsx')
df = pd.read_excel(file, parse_dates=True, index_col='Start Date')

# read in OEE losses
file = (dir + 'AL6_OEE_Losses.xlsx')
df_OEE = pd.read_excel(file)
df_OEE.rename(columns=({'Calendar Day':'Date', 
                        'Point of OEE':'OEE',
                        'Loss Category':'Category'}), inplace=True)
df_OEE['OEE'] = pd.to_numeric(df_OEE['OEE'], errors='coerce')
df_OEE['OEE'] = df_OEE['OEE'].mul(100)
df_OEE = df_OEE.set_index('Date')
df_OEE.shape

(2687, 2)

get rid of any pre April 2021 data - wasn't valid for AL6

In [3]:
start_date = '2021-04-01'
end_date = datetime.date.today()
df_OEE = df_OEE[df_OEE.index >= start_date].sort_values('Date')
df_OEE.Category.unique()

array(['Qualifizierung/Validierung (11)', 'Production', 'Minor Loss',
       'Without cause', 'Sonstiges und Unbekannte (26)',
       'Störungen und Fehlerbehebung (25)', 'Speed Loss',
       'W&I (geplante Linie) (23)',
       'Organisatorische Verluste und Wartezeiten (22)',
       'Rüstvorgänge (21)', 'Ungeplante Linie (14)',
       'Feier/Brückentag (12)', 'Autonome Wartungen (24)',
       'vorbeugende Instandhaltung/Wartung (13)', 'Sonstige (15)',
       'Quality Loss'], dtype=object)

Let's translate these categories!

In [4]:
df_OEE['Category'].replace({
'Qualifizierung/Validierung (11)':'Qualification/validation', 
'Sonstiges und Unbekannte (26)':'Other and unknown',
'Störungen und Fehlerbehebung (25)':'Malfunctions and troubleshooting', 
'W&I (geplante Linie) (23)':'W&I (planned line)',
'Organisatorische Verluste und Wartezeiten (22)':'Organizational losses and waiting times',
'Rüstvorgänge (21)':'Changeover', 
'Ungeplante Linie (14)':'Unplanned Line',
'Feier/Brückentag (12)':'Celebration / long day', 
'Autonome Wartungen (24)':'Autonomous maintenance',
'vorbeugende Instandhaltung/Wartung (13)':'preventive maintenance / servicing', 
'Sonstige (15)':'Other'}, inplace=True)

In [5]:
# set up date range with min start_date and max start_date recorded earlier
daterange = pd.date_range(start = start_date, end = end_date, freq='D')

pd.DataFrame({'Date': daterange.date})

# df = pd.merge(df1, df, on=['Date','UI'], how='outer')

,Date
0,2021-04-01
1,2021-04-02
2,2021-04-03
3,2021-04-04
4,2021-04-05
...,...
101,2021-07-11
102,2021-07-12
103,2021-07-13
104,2021-07-14


In [6]:
df_OEE['Month'] = df_OEE.index.strftime("%B")
df_OEE['Day'] = df_OEE.index.strftime("%d")
df_OEE.head()

# df_OEE.groupby(['Day', 'Month', 'Category'])[['OEE']].sum()

df_OEE['Month'] = df_OEE['Month'].astype(pd.api.types.CategoricalDtype(categories=['April','May','June', 'July'], ordered=True))

production = df_OEE[df_OEE.Category.str.contains('Production')]
changeover = df_OEE[df_OEE.Category.str.contains('21')]

pivot = production.pivot(index='Day', columns='Month', values='OEE').style.background_gradient(cmap='Oranges')\
    .format("{:.2f}")

pivot

Month,April,May,June,July
Day,,,,
01,43.38,43.10,54.36,44.02
02,16.43,38.67,21.29,31.94
03,61.40,29.59,44.45,17.03
04,0.01,30.71,80.18,0.00
05,nan,32.49,53.34,nan
06,27.23,33.51,18.67,nan
07,33.66,67.48,29.44,nan
08,50.58,60.34,39.22,nan
09,49.16,41.13,25.17,51.37


In [7]:
pivot.to_excel(dir + "AL6_production_losses_pivot.xlsx")

In [10]:
writer = pd.ExcelWriter('AL6_OEE_Losses_pivot.xlsx', engine='xlsxwriter')
for i in df_OEE.Month.unique():
    pivot_OEE = df_OEE.pivot(index='Day', columns=['Category','Month'], values='OEE').filter(regex=i).style.background_gradient(cmap='Oranges')
    pivot_OEE.to_excel(writer, sheet_name=i)

workbook = writer.book
cell_format = workbook.add_format()
decimal = workbook.add_format({'num_format': '0.00'})
writer.save()

ModuleNotFoundError: No module named 'xlsxwriter'

In [224]:
df_grouped = df.groupby([pd.Grouper(freq='D'), 'Machine'])['Duration (min.)'].sum().reset_index()
pivot = df_grouped.pivot(index='Start Date', columns='Machine', values='Duration (min.)')
pivot.fillna(0, inplace=True)
pivot['Total Duration'] = pivot.sum(axis=1)
pivot.nlargest(15, 'Total Duration').style.background_gradient(cmap='Purples')


Machine,AL6 Depalettierer,AL6 FS Bedruckung,AL6 Kartonierer,AL6 Kontrollwaage,Total Duration
Start Date,,,,,
2021-04-10 00:00:00,0.000000,19.516667,1151.666667,0.000000,1171.183333
2021-05-04 00:00:00,53.783333,6.400000,786.683333,0.000000,846.866667
2021-04-12 00:00:00,0.000000,0.000000,725.416667,0.000000,725.416667
2021-04-09 00:00:00,12.600000,82.783333,479.166667,0.000000,574.550000
2021-04-27 00:00:00,19.416667,21.700000,511.900000,0.000000,553.016667
2021-04-29 00:00:00,18.600000,88.633333,352.800000,0.000000,460.033333
2021-05-29 00:00:00,0.000000,124.483333,314.933333,0.000000,439.416667
2021-05-05 00:00:00,10.916667,39.916667,382.533333,0.000000,433.366667
2021-05-28 00:00:00,0.000000,30.833333,400.466667,0.000000,431.300000


In [5]:
df.loc['2021-04-10'].groupby(['Machine','Stoppage Reason 3','Last Level Reason']).sum().sort_values('Duration (min.)', ascending=False)

Duration (min.)
Machine           Stoppage Reason 3                     Last Level Reason                                      
AL6 Kartonierer   Störungen und Fehlerbehebung (25)     F2_202- Werkzeug (251441104)                 537.216667
                                                        F3_304- Pneumatik (251441505)                327.133333
                  Minor Loss                            -                                             77.416667
                  Störungen und Fehlerbehebung (25)     Steppkette 1 (251440814)                      36.933333
                                                        F3_324- Werkzeug (251441512)                  34.516667
                  Sonstige/ Unbekannte Stillstände (26) Sonstige/ Unbekannte Stillstände (261)        28.833333
                  Störungen und Fehlerbehebung (25)     Kamera L6 (2512902)                           26.833333
                                                        F3_304- Werkzeug (251441508)                  24.250000
                                                        F2_204- Werkzeug (251441504)                  15.683333
                                                        Faltschachtelmagazin (251441110)              14.833333
AL6 FS Bedruckung Störungen und Fehlerbehebung (25)     Kontrolle Variable Daten (2512606)            12.383333
AL6 Kartonierer   Störungen und Fehlerbehebung (25)     F2_224- Abholwerkzeug PENs (251440803)         7.083333
                                                        Druckeinheit (2512601)                         6.983333
                  Without cause                         Without cause                                  6.550000
AL6 FS Bedruckung Autonome Wartungen (24)               Etikettenaufroller (2422603)                   5.000000
AL6 Kartonierer   Autonome Wartungen (24)               geplante Wartung (Operator) (24104)            4.666667
                  Störungen und Fehlerbehebung (25)     F4_401- Werkzeug (251440604)                   2.733333
AL6 FS Bedruckung Autonome Wartungen (24)               Kontrolle Variable Daten (2422606)             2.133333

In [13]:
df[df.Machine.str.contains('Kart')].loc['2021-04-10'].sort_values('Duration (min.)', ascending=False)

,Shift Long Name,End Date,Duration (min.),Stoppage Reason 3,Last Level Reason,Machine,Comments,Creator
Start Date,,,,,,,,
2021-04-10 05:42:48,NACH - 09/04/2021 22:00:00,2021-04-10 10:06:13,263.416667,Störungen und Fehlerbehebung (25),F2_202- Werkzeug (251441104),AL6 Kartonierer,-,DE013273
2021-04-10 05:42:48,FRUE - 10/04/2021 06:00:00,2021-04-10 10:06:13,263.416667,Störungen und Fehlerbehebung (25),F2_202- Werkzeug (251441104),AL6 Kartonierer,-,DE013273
2021-04-10 12:18:03,FRUE - 10/04/2021 06:00:00,2021-04-10 15:01:37,163.566667,Störungen und Fehlerbehebung (25),F3_304- Pneumatik (251441505),AL6 Kartonierer,---,MIIPCOPQH
2021-04-10 12:18:03,SPAE - 10/04/2021 14:00:00,2021-04-10 15:01:37,163.566667,Störungen und Fehlerbehebung (25),F3_304- Pneumatik (251441505),AL6 Kartonierer,---,MIIPCOPQH
2021-04-10 19:31:51,SPAE - 10/04/2021 14:00:00,2021-04-10 20:08:47,36.933333,Störungen und Fehlerbehebung (25),Steppkette 1 (251440814),AL6 Kartonierer,-,DE016909
...,...,...,...,...,...,...,...,...
2021-04-10 20:15:37,SPAE - 10/04/2021 14:00:00,2021-04-10 20:15:52,0.250000,Minor Loss,-,AL6 Kartonierer,---,MIIPCOPQH
2021-04-10 21:47:21,SPAE - 10/04/2021 14:00:00,2021-04-10 21:47:36,0.250000,Minor Loss,-,AL6 Kartonierer,---,MIIPCOPQH
2021-04-10 19:07:57,SPAE - 10/04/2021 14:00:00,2021-04-10 19:08:11,0.233333,Minor Loss,-,AL6 Kartonierer,---,MIIPCOPQH
